In [ ]:
from dolfin import *
import numpy as np
import matplotlib.pyplot as plt
#%config InlineBackend.figure_format = 'retina'

In [ ]:
def curl_curl(n, r=1):    
    mesh = UnitSquareMesh(n, n)
    
    # start by solving for u using non-hybrid method
    S_element = FiniteElement('CG', mesh.ufl_cell(), r+1)
    V_element = FiniteElement('N2curl', mesh.ufl_cell(), r)
    W = FunctionSpace(mesh, S_element*V_element)
    
    bcs = [DirichletBC(W.sub(0), Constant(0), DomainBoundary()),
           DirichletBC(W.sub(1), Constant((0,0)), DomainBoundary())]

    s, u = TrialFunctions(W)
    t, v = TestFunctions(W) 
    a = (-inner(s, t) + inner(u, grad(t)) + inner(grad(s), v) + inner(curl(u), curl(v)))*dx
    #for q = curl u to match other formulations
    f = Expression(('-pow(pi,2)*sin(pi*x[1])','pow(pi,2)*sin(pi*x[0])'), degree=r+1)
    L = inner(f, v)*dx
    
    w = Function(W)
    solve(a == L, w, bcs)
    u = w.sub(1)

    curl_u = project(curl(u), FunctionSpace(mesh, "DG", r-1))
    
    # post-process
    V_dg_element = VectorElement('DG', mesh.ufl_cell(), r)
    Q_element = FiniteElement('CG', mesh.ufl_cell(), r+2)
    W = FunctionSpace(mesh, V_dg_element*Q_element)
    
    lam, q = TrialFunctions(W)
    v, p = TestFunctions(W)
    a = (inner(q, p) + inner(curl(q), curl(p)) + curl(lam*p + v*q) + 1e-10*inner(lam, v))*dx
    L = (inner(f, v + curl(p)) + inner(curl_u, p + curl(v)))*dx
    
    w = Function(W)
    solve(a == L, w)

    return u, w.sub(1)

def grad_div_rt(n, r=1):
    mesh = UnitSquareMesh(n, n)
    
    # start by solving for u using non-hybrid method
    S_element = FiniteElement('CG', mesh.ufl_cell(), r)
    V_element = FiniteElement('RT', mesh.ufl_cell(), r)
    W = FunctionSpace(mesh, S_element*V_element)
    bcs = [DirichletBC(W.sub(0), Constant(0), DomainBoundary()),
           DirichletBC(W.sub(1), Constant((0,0)), DomainBoundary())]

    s, u = TrialFunctions(W)
    t, v = TestFunctions(W)
    a = (-inner(s, t) + inner(u, curl(t)) + inner(curl(s), v) + inner(div(u), div(v)))*dx
    f = Expression(("pow(pi,2)*sin(pi*x[0])","pow(pi,2)*sin(pi*x[1])"), degree=r+1)
    L = inner(f, v)*dx
    
    w = Function(W)
    solve(a == L, w, bcs)
    u = w.sub(1)

    div_u = project(div(u), FunctionSpace(mesh, "DG", r-1))
    
    # post-process to estimate q = div(u) in H^1 instead of L^2
    V_dg_element = FiniteElement('DRT', mesh.ufl_cell(), r)
    Q_element = FiniteElement('CG', mesh.ufl_cell(), r+1)
    W = FunctionSpace(mesh, V_dg_element*Q_element)
    
    lam, q = TrialFunctions(W)
    v, p = TestFunctions(W)
    a = (inner(q, p) + inner(grad(q), grad(p)) - div(lam*p + v*q) + 1e-10*inner(lam, v))*dx
    L = (inner(f, v - grad(p)) + inner(div_u, p - div(v)))*dx
    
    w = Function(W)
    solve(a == L, w)

    return u, w.sub(1)

def grad_div_bdm(n, r=1):
    mesh = UnitSquareMesh(n, n)
    
    # start by solving for u using non-hybrid method
    S_element = FiniteElement('CG', mesh.ufl_cell(), r+1)
    V_element = FiniteElement('BDM', mesh.ufl_cell(), r)
    W = FunctionSpace(mesh, S_element*V_element)
    bcs = [DirichletBC(W.sub(0), Constant(0), DomainBoundary()),
           DirichletBC(W.sub(1), Constant((0,0)), DomainBoundary())]

    s, u = TrialFunctions(W)
    t, v = TestFunctions(W)
    a = (-inner(s, t) + inner(u, curl(t)) + inner(curl(s), v) + inner(div(u), div(v)))*dx
    f = Expression(("pow(pi,2)*sin(pi*x[0])","pow(pi,2)*sin(pi*x[1])"), degree=r+1)
    L = inner(f, v)*dx
    
    w = Function(W)
    solve(a == L, w, bcs)
    u = w.sub(1)

    div_u = project(div(u), FunctionSpace(mesh, "DG", r-1))
    
    # post-process to estimate q = div(u) in H^1 instead of L^2
    V_dg_element = VectorElement('DG', mesh.ufl_cell(), r)
    Q_element = FiniteElement('CG', mesh.ufl_cell(), r+2)
    W = FunctionSpace(mesh, V_dg_element*Q_element)
    
    lam, q = TrialFunctions(W)
    v, p = TestFunctions(W)
    a = (inner(q, p) + inner(grad(q), grad(p)) - div(lam*p + v*q) + 1e-10*inner(lam, v))*dx
    L = (inner(f, v - grad(p)) + inner(div_u, p - div(v)))*dx
    
    w = Function(W)
    solve(a == L, w)

    return u, w.sub(1)


In [ ]:
degrees = range(1,4)
ns = [2**k for k in range(1,3)]

num_ns = len(ns)
num_dgs = len(degrees)

u_es = []
q_es = []

errors = {}

for degree in degrees:
    print(degree)
    es = []
    for i, n in enumerate(ns):
        u_cc,  q_cc  = curl_curl(n, r=degree)
        u_rt,  q_rt  = grad_div_rt(n, r=degree)
        u_bdm, q_bdm = grad_div_bdm(n, r=degree)
        u_es.append([errornorm(u_cc, u_rt), errornorm(u_cc, u_bdm)])
        q_es.append([errornorm(q_cc, q_rt), errornorm(q_cc, q_bdm)])

        u_exact_cc = Expression(("-sin(pi*x[0])","sin(pi*x[1])"), degree=degree+3)
        u_exact = Expression(("sin(pi*x[0])","sin(pi*x[1])"), degree=degree+3)
        q_exact = Expression("pi*(cos(pi*x[0]) + cos(pi*x[1]))", degree=degree+4)

        es.append([errornorm(q_exact, q_cc),
                       errornorm(q_exact, q_rt),
                       errornorm(q_exact, q_bdm)])#,
        """
                       errornorm(u_exact_cc ,u_cc),
                       errornorm(u_exact, u_rt),
                       errornorm(u_exact, u_bdm)])
        """

        '''
        # see that q_cc is rotated 90 version of q_rt & q_bdm
        plt.subplot(num_dgs*num_ns, 3, 3*(i+num_ns*(degree-1))+1)
        plt.title("q_cc, r=%d"%degree)
        a = plot(q_cc)
        plt.colorbar(a)
        plt.subplot(num_dgs*num_ns, 3, 3*(i+num_ns*(degree-1))+2)
        plt.title("q_rt, r=%d"%degree)
        b = plot(q_rt)
        plt.colorbar(b)
        plt.subplot(num_dgs*num_ns, 3, 3*(i+num_ns*(degree-1))+3)
        plt.title("q_bdm, r=%d"%degree)
        c = plot(q_bdm)
        plt.colorbar(c)
        '''
    errors[degree] = es
plt.subplots_adjust(wspace=0.5, hspace=.4, left=0.01, right=2.0, bottom=0, top=4)

In [ ]:
q_es

In [ ]:
u_es

In [ ]:

for degree in degrees:
    print(errors[degree])
    plt.subplot(num_dgs, 1, degree)
    plt.loglog(1/np.array(ns), errors[degree], basex=2, basey=2)
    plt.title("degree = %d"%degree)
    plt.legend(["q_cc", "q_rt", "q_bdm"])#, "u_cc", "u_rt", "u_bdm"])

plt.subplots_adjust(wspace=0.5, hspace=.4, left=0.01, right=2.0, bottom=0, top=4)